In [1]:
import sympy as sp

In [2]:
def quaternionMatrix(q):
    q0, q1, q2, q3 = q
    # First row of the rotation matrix
    r00 = 2 * (q0 * q0 + q1 * q1) - 1
    r01 = 2 * (q1 * q2 - q0 * q3)
    r02 = 2 * (q1 * q3 + q0 * q2)
     
    # Second row of the rotation matrix
    r10 = 2 * (q1 * q2 + q0 * q3)
    r11 = 2 * (q0 * q0 + q2 * q2) - 1
    r12 = 2 * (q2 * q3 - q0 * q1)
     
    # Third row of the rotation matrix
    r20 = 2 * (q1 * q3 - q0 * q2)
    r21 = 2 * (q2 * q3 + q0 * q1)
    r22 = 2 * (q0 * q0 + q3 * q3) - 1
     
    # 3x3 rotation matrix
    rot_matrix = sp.Matrix([[r00, r01, r02],
                           [r10, r11, r12],
                           [r20, r21, r22]])
                            
    return rot_matrix

def rpyMatrix(x, y, z):
    # Roll matrix
    roll_matrix = sp.Matrix([[1, 0, 0],
                             [0, sp.cos(x), -sp.sin(x)],
                             [0, sp.sin(x), sp.cos(x)]])
     
    # Pitch matrix
    pitch_matrix = sp.Matrix([[sp.cos(y), 0, sp.sin(y)],
                              [0, 1, 0],
                              [-sp.sin(y), 0, sp.cos(y)]])
     
    # Yaw matrix
    yaw_matrix = sp.Matrix([[sp.cos(z), -sp.sin(z), 0],
                            [sp.sin(z), sp.cos(z), 0],
                            [0, 0, 1]])
     
    # ZYX Rotation matrix
    rot_matrix = yaw_matrix * pitch_matrix * roll_matrix
     
    return rot_matrix

def rotationMatrix():
    r00, r01, r02 = sp.symbols('r00 r01 r02')
    r10, r11, r12 = sp.symbols('r10 r11 r12')
    r20, r21, r22 = sp.symbols('r20 r21 r22')
    # 3x3 rotation matrix
    rot_matrix = sp.Matrix([[r00, r01, r02],
                           [r10, r11, r12],
                           [r20, r21, r22]])
                            
    return rot_matrix

def matrix2rpy(R):
    r11, r12, r13 = R[0, :]
    r21, r22, r23 = R[1, :]
    r31, r32, r33 = R[2, :]
    roll = sp.atan2(r32, r33)
    pitch = sp.atan2(-r31, sp.sqrt(r32**2 + r33**2))
    yaw = sp.atan2(r21, r11)
    return roll, pitch, yaw



In [3]:
#q0, q1, q2, q3 = sp.symbols('q0 q1 q2 q3')
#q = [q0, q1, q2, q3]
#R = quaternionMatrix(q)
R = rotationMatrix()
R_vec = sp.Matrix([R[0, 0], R[0, 1], R[0, 2], R[1, 0], R[1, 1], R[1, 2], R[2, 0], R[2, 1], R[2, 2]])
display(R_vec)
display(R)
rx, ry, rz = sp.symbols('rx ry rz')
Rrpy = rpyMatrix(rx, ry, rz)
display(Rrpy)

Matrix([
[r00],
[r01],
[r02],
[r10],
[r11],
[r12],
[r20],
[r21],
[r22]])

Matrix([
[r00, r01, r02],
[r10, r11, r12],
[r20, r21, r22]])

Matrix([
[cos(ry)*cos(rz), sin(rx)*sin(ry)*cos(rz) - sin(rz)*cos(rx),  sin(rx)*sin(rz) + sin(ry)*cos(rx)*cos(rz)],
[sin(rz)*cos(ry), sin(rx)*sin(ry)*sin(rz) + cos(rx)*cos(rz), -sin(rx)*cos(rz) + sin(ry)*sin(rz)*cos(rx)],
[       -sin(ry),                           sin(rx)*cos(ry),                            cos(rx)*cos(ry)]])

In [4]:
# Settings
## Force
f1, f2, f3 = sp.symbols('f1 f2 f3')
th1, th2, th3 = sp.symbols('th1 th2 th3')
F1_l = sp.Matrix([0, sp.sin(th1)*f1, sp.cos(th1)*f1])
F2_l = sp.Matrix([0, sp.sin(th2)*f2, sp.cos(th2)*f2])
F3_l = sp.Matrix([0, sp.sin(th3)*f3, sp.cos(th3)*f3])
m = 0.479 # kg
## Length
r = 0.323 # m
s1_l = sp.Matrix([r,0,0])
# degrees to radians
rad120 = 120 * sp.pi / 180
F1_l = rpyMatrix(0,0,rad120) @ F1_l
s2_l = sp.Matrix([r*sp.cos(rad120),r*sp.sin(rad120),0])
rad240 = 240 * sp.pi / 180
F2_l = rpyMatrix(0,0,rad240) @ F2_l
s3_l = sp.Matrix([r*sp.cos(rad240),r*sp.sin(rad240),0])

## Inertia
#<inertia  ixx="12.108167932855986" ixy="0" ixz="0" iyy="15.738311498959863" iyz="0" izz="27.18371465461616" />
I_l = sp.Matrix([[12.108167932855986,0,0],[0,15.738311498959863,0],[0,0,27.18371465461616]])

## Dispaly
display(F1_l)
display(F2_l)
display(F3_l)

Matrix([
[-sqrt(3)*f1*sin(th1)/2],
[        -f1*sin(th1)/2],
[           f1*cos(th1)]])

Matrix([
[sqrt(3)*f2*sin(th2)/2],
[       -f2*sin(th2)/2],
[          f2*cos(th2)]])

Matrix([
[          0],
[f3*sin(th3)],
[f3*cos(th3)]])

In [5]:
# Force
F1 = Rrpy @ F1_l
F2 = Rrpy @ F2_l
F3 = Rrpy @ F3_l
g = sp.Matrix([0,0,-9.81])
F_gravity = m*g

print(F1)
print(F2)
print(F3)
print("Total force:")
F = F1+F2+F3+F_gravity
F = sp.simplify(F)
display(F)

Matrix([[f1*(sin(rx)*sin(rz) + sin(ry)*cos(rx)*cos(rz))*cos(th1) - f1*(sin(rx)*sin(ry)*cos(rz) - sin(rz)*cos(rx))*sin(th1)/2 - sqrt(3)*f1*sin(th1)*cos(ry)*cos(rz)/2], [f1*(-sin(rx)*cos(rz) + sin(ry)*sin(rz)*cos(rx))*cos(th1) - f1*(sin(rx)*sin(ry)*sin(rz) + cos(rx)*cos(rz))*sin(th1)/2 - sqrt(3)*f1*sin(rz)*sin(th1)*cos(ry)/2], [-f1*sin(rx)*sin(th1)*cos(ry)/2 + sqrt(3)*f1*sin(ry)*sin(th1)/2 + f1*cos(rx)*cos(ry)*cos(th1)]])
Matrix([[f2*(sin(rx)*sin(rz) + sin(ry)*cos(rx)*cos(rz))*cos(th2) - f2*(sin(rx)*sin(ry)*cos(rz) - sin(rz)*cos(rx))*sin(th2)/2 + sqrt(3)*f2*sin(th2)*cos(ry)*cos(rz)/2], [f2*(-sin(rx)*cos(rz) + sin(ry)*sin(rz)*cos(rx))*cos(th2) - f2*(sin(rx)*sin(ry)*sin(rz) + cos(rx)*cos(rz))*sin(th2)/2 + sqrt(3)*f2*sin(rz)*sin(th2)*cos(ry)/2], [-f2*sin(rx)*sin(th2)*cos(ry)/2 - sqrt(3)*f2*sin(ry)*sin(th2)/2 + f2*cos(rx)*cos(ry)*cos(th2)]])
Matrix([[f3*(sin(rx)*sin(rz) + sin(ry)*cos(rx)*cos(rz))*cos(th3) + f3*(sin(rx)*sin(ry)*cos(rz) - sin(rz)*cos(rx))*sin(th3)], [f3*(-sin(rx)*cos(rz) + sin

Matrix([
[-f1*sin(rx)*sin(ry)*sin(th1)*cos(rz)/2 + f1*sin(rx)*sin(rz)*cos(th1) + f1*sin(ry)*cos(rx)*cos(rz)*cos(th1) + f1*sin(rz)*sin(th1)*cos(rx)/2 - sqrt(3)*f1*sin(th1)*cos(ry)*cos(rz)/2 - f2*sin(rx)*sin(ry)*sin(th2)*cos(rz)/2 + f2*sin(rx)*sin(rz)*cos(th2) + f2*sin(ry)*cos(rx)*cos(rz)*cos(th2) + f2*sin(rz)*sin(th2)*cos(rx)/2 + sqrt(3)*f2*sin(th2)*cos(ry)*cos(rz)/2 + f3*sin(ry)*cos(rz)*cos(rx - th3) + f3*sin(rz)*sin(rx - th3)],
[-f1*sin(rx)*sin(ry)*sin(rz)*sin(th1)/2 - f1*sin(rx)*cos(rz)*cos(th1) + f1*sin(ry)*sin(rz)*cos(rx)*cos(th1) - sqrt(3)*f1*sin(rz)*sin(th1)*cos(ry)/2 - f1*sin(th1)*cos(rx)*cos(rz)/2 - f2*sin(rx)*sin(ry)*sin(rz)*sin(th2)/2 - f2*sin(rx)*cos(rz)*cos(th2) + f2*sin(ry)*sin(rz)*cos(rx)*cos(th2) + sqrt(3)*f2*sin(rz)*sin(th2)*cos(ry)/2 - f2*sin(th2)*cos(rx)*cos(rz)/2 + f3*sin(ry)*sin(rz)*cos(rx - th3) - f3*sin(rx - th3)*cos(rz)],
[                                                                                                                                              

In [6]:
# Linear acceleration
a = F/m
a = sp.simplify(a)

print("Linear Acceleration:")
display(a)


Linear Acceleration:


Matrix([
[-1.04384133611691*f1*sin(rx)*sin(ry)*sin(th1)*cos(rz) + 2.08768267223382*f1*sin(rx)*sin(rz)*cos(th1) + 2.08768267223382*f1*sin(ry)*cos(rx)*cos(rz)*cos(th1) + 1.04384133611691*f1*sin(rz)*sin(th1)*cos(rx) - 1.04384133611691*sqrt(3)*f1*sin(th1)*cos(ry)*cos(rz) - 1.04384133611691*f2*sin(rx)*sin(ry)*sin(th2)*cos(rz) + 2.08768267223382*f2*sin(rx)*sin(rz)*cos(th2) + 2.08768267223382*f2*sin(ry)*cos(rx)*cos(rz)*cos(th2) + 1.04384133611691*f2*sin(rz)*sin(th2)*cos(rx) + 1.04384133611691*sqrt(3)*f2*sin(th2)*cos(ry)*cos(rz) + 2.08768267223382*f3*sin(ry)*cos(rz)*cos(rx - th3) + 2.08768267223382*f3*sin(rz)*sin(rx - th3)],
[-1.04384133611691*f1*sin(rx)*sin(ry)*sin(rz)*sin(th1) - 2.08768267223382*f1*sin(rx)*cos(rz)*cos(th1) + 2.08768267223382*f1*sin(ry)*sin(rz)*cos(rx)*cos(th1) - 1.04384133611691*sqrt(3)*f1*sin(rz)*sin(th1)*cos(ry) - 1.04384133611691*f1*sin(th1)*cos(rx)*cos(rz) - 1.04384133611691*f2*sin(rx)*sin(ry)*sin(rz)*sin(th2) - 2.08768267223382*f2*sin(rx)*cos(rz)*cos(th2) + 2.0876826722

In [7]:
# Find linear position
t = sp.symbols('t')
v0x, v0y, v0z = sp.symbols('v0x v0y v0z')
p0x, p0y, p0z = sp.symbols('p0x p0y p0z')
v0 = sp.Matrix([v0x,v0y,v0z])
p0 = sp.Matrix([p0x,p0y,p0z])
v = v0 + sp.integrate(a, t)
v = sp.simplify(v)
p = p0 + sp.integrate(v, t)
p = sp.simplify(p)
print("Linear Velocity:")
display(v)
print("Linear Position:")
display(p)

Linear Velocity:


Matrix([
[t*(-1.04384133611691*f1*sin(rx)*sin(ry)*sin(th1)*cos(rz) + 2.08768267223382*f1*sin(rx)*sin(rz)*cos(th1) + 2.08768267223382*f1*sin(ry)*cos(rx)*cos(rz)*cos(th1) + 1.04384133611691*f1*sin(rz)*sin(th1)*cos(rx) - 1.04384133611691*sqrt(3)*f1*sin(th1)*cos(ry)*cos(rz) - 1.04384133611691*f2*sin(rx)*sin(ry)*sin(th2)*cos(rz) + 2.08768267223382*f2*sin(rx)*sin(rz)*cos(th2) + 2.08768267223382*f2*sin(ry)*cos(rx)*cos(rz)*cos(th2) + 1.04384133611691*f2*sin(rz)*sin(th2)*cos(rx) + 1.04384133611691*sqrt(3)*f2*sin(th2)*cos(ry)*cos(rz) + 2.08768267223382*f3*sin(ry)*cos(rz)*cos(rx - th3) + 2.08768267223382*f3*sin(rz)*sin(rx - th3)) + v0x],
[-t*(1.04384133611691*f1*sin(rx)*sin(ry)*sin(rz)*sin(th1) + 2.08768267223382*f1*sin(rx)*cos(rz)*cos(th1) - 2.08768267223382*f1*sin(ry)*sin(rz)*cos(rx)*cos(th1) + 1.04384133611691*sqrt(3)*f1*sin(rz)*sin(th1)*cos(ry) + 1.04384133611691*f1*sin(th1)*cos(rx)*cos(rz) + 1.04384133611691*f2*sin(rx)*sin(ry)*sin(rz)*sin(th2) + 2.08768267223382*f2*sin(rx)*cos(rz)*cos(th2) -

Linear Position:


Matrix([
[p0x + t**2*(-0.521920668058455*f1*sin(rx)*sin(ry)*sin(th1)*cos(rz) + 1.04384133611691*f1*sin(rx)*sin(rz)*cos(th1) + 1.04384133611691*f1*sin(ry)*cos(rx)*cos(rz)*cos(th1) + 0.521920668058455*f1*sin(rz)*sin(th1)*cos(rx) - 0.521920668058455*sqrt(3)*f1*sin(th1)*cos(ry)*cos(rz) - 0.521920668058455*f2*sin(rx)*sin(ry)*sin(th2)*cos(rz) + 1.04384133611691*f2*sin(rx)*sin(rz)*cos(th2) + 1.04384133611691*f2*sin(ry)*cos(rx)*cos(rz)*cos(th2) + 0.521920668058455*f2*sin(rz)*sin(th2)*cos(rx) + 0.521920668058455*sqrt(3)*f2*sin(th2)*cos(ry)*cos(rz) + 1.04384133611691*f3*sin(ry)*cos(rz)*cos(rx - th3) + 1.04384133611691*f3*sin(rz)*sin(rx - th3)) + t*v0x],
[p0y + t**2*(-0.521920668058455*f1*sin(rx)*sin(ry)*sin(rz)*sin(th1) - 1.04384133611691*f1*sin(rx)*cos(rz)*cos(th1) + 1.04384133611691*f1*sin(ry)*sin(rz)*cos(rx)*cos(th1) - 0.521920668058455*sqrt(3)*f1*sin(rz)*sin(th1)*cos(ry) - 0.521920668058455*f1*sin(th1)*cos(rx)*cos(rz) - 0.521920668058455*f2*sin(rx)*sin(ry)*sin(rz)*sin(th2) - 1.04384133611691

In [8]:
I_inv = R @ I_l.inv() @ R.T
I_inv = sp.simplify(I_inv)
display(I_inv)

Matrix([
[   0.082588877652288*r00**2 + 0.0635392176642386*r01**2 + 0.0367867310522326*r02**2, 0.082588877652288*r00*r10 + 0.0635392176642386*r01*r11 + 0.0367867310522326*r02*r12, 0.082588877652288*r00*r20 + 0.0635392176642386*r01*r21 + 0.0367867310522326*r02*r22],
[0.082588877652288*r00*r10 + 0.0635392176642386*r01*r11 + 0.0367867310522326*r02*r12,    0.082588877652288*r10**2 + 0.0635392176642386*r11**2 + 0.0367867310522326*r12**2, 0.082588877652288*r10*r20 + 0.0635392176642386*r11*r21 + 0.0367867310522326*r12*r22],
[0.082588877652288*r00*r20 + 0.0635392176642386*r01*r21 + 0.0367867310522326*r02*r22, 0.082588877652288*r10*r20 + 0.0635392176642386*r11*r21 + 0.0367867310522326*r12*r22,    0.082588877652288*r20**2 + 0.0635392176642386*r21**2 + 0.0367867310522326*r22**2]])

In [9]:
F1 = R @ F1_l
F2 = R @ F2_l
F3 = R @ F3_l

# Torque
s1 = R @ s1_l
s2 = R @ s2_l
s3 = R @ s3_l

tau1 = s1.cross(F1)
tau2 = s2.cross(F2)
tau3 = s3.cross(F3)

print(tau1)
print(tau2)
print(tau3)

print("Total torque:")
M = tau1+tau2+tau3
M = sp.simplify(M)
display(M)

Matrix([[0.323*r10*(-sqrt(3)*f1*r20*sin(th1)/2 - f1*r21*sin(th1)/2 + f1*r22*cos(th1)) - 0.323*r20*(-sqrt(3)*f1*r10*sin(th1)/2 - f1*r11*sin(th1)/2 + f1*r12*cos(th1))], [-0.323*r00*(-sqrt(3)*f1*r20*sin(th1)/2 - f1*r21*sin(th1)/2 + f1*r22*cos(th1)) + 0.323*r20*(-sqrt(3)*f1*r00*sin(th1)/2 - f1*r01*sin(th1)/2 + f1*r02*cos(th1))], [0.323*r00*(-sqrt(3)*f1*r10*sin(th1)/2 - f1*r11*sin(th1)/2 + f1*r12*cos(th1)) - 0.323*r10*(-sqrt(3)*f1*r00*sin(th1)/2 - f1*r01*sin(th1)/2 + f1*r02*cos(th1))]])
Matrix([[(-0.1615*r10 + 0.1615*sqrt(3)*r11)*(sqrt(3)*f2*r20*sin(th2)/2 - f2*r21*sin(th2)/2 + f2*r22*cos(th2)) - (-0.1615*r20 + 0.1615*sqrt(3)*r21)*(sqrt(3)*f2*r10*sin(th2)/2 - f2*r11*sin(th2)/2 + f2*r12*cos(th2))], [-(-0.1615*r00 + 0.1615*sqrt(3)*r01)*(sqrt(3)*f2*r20*sin(th2)/2 - f2*r21*sin(th2)/2 + f2*r22*cos(th2)) + (-0.1615*r20 + 0.1615*sqrt(3)*r21)*(sqrt(3)*f2*r00*sin(th2)/2 - f2*r01*sin(th2)/2 + f2*r02*cos(th2))], [(-0.1615*r00 + 0.1615*sqrt(3)*r01)*(sqrt(3)*f2*r10*sin(th2)/2 - f2*r11*sin(th2)/2 + f2*r1

Matrix([
[-0.1615*f1*r10*(sqrt(3)*r20*sin(th1) + r21*sin(th1) - 2*r22*cos(th1)) + 0.1615*f1*r20*(sqrt(3)*r10*sin(th1) + r11*sin(th1) - 2*r12*cos(th1)) - 0.08075*f2*(r10 - sqrt(3)*r11)*(sqrt(3)*r20*sin(th2) - r21*sin(th2) + 2*r22*cos(th2)) + 0.08075*f2*(r20 - sqrt(3)*r21)*(sqrt(3)*r10*sin(th2) - r11*sin(th2) + 2*r12*cos(th2)) - 0.1615*f3*(r10 + sqrt(3)*r11)*(r21*sin(th3) + r22*cos(th3)) + 0.1615*f3*(r20 + sqrt(3)*r21)*(r11*sin(th3) + r12*cos(th3))],
[ 0.1615*f1*r00*(sqrt(3)*r20*sin(th1) + r21*sin(th1) - 2*r22*cos(th1)) - 0.1615*f1*r20*(sqrt(3)*r00*sin(th1) + r01*sin(th1) - 2*r02*cos(th1)) + 0.08075*f2*(r00 - sqrt(3)*r01)*(sqrt(3)*r20*sin(th2) - r21*sin(th2) + 2*r22*cos(th2)) - 0.08075*f2*(r20 - sqrt(3)*r21)*(sqrt(3)*r00*sin(th2) - r01*sin(th2) + 2*r02*cos(th2)) + 0.1615*f3*(r00 + sqrt(3)*r01)*(r21*sin(th3) + r22*cos(th3)) - 0.1615*f3*(r20 + sqrt(3)*r21)*(r01*sin(th3) + r02*cos(th3))],
[-0.1615*f1*r00*(sqrt(3)*r10*sin(th1) + r11*sin(th1) - 2*r12*cos(th1)) + 0.1615*f1*r10*(sqrt(3)*r00*sin

In [10]:
# Angular acceleration
alpha = I_inv @ M
alpha = sp.simplify(alpha)

print("Angular Acceleration:")
display(alpha)

Angular Acceleration:


Matrix([
[-(0.082588877652288*r00**2 + 0.0635392176642386*r01**2 + 0.0367867310522326*r02**2)*(0.1615*f1*r10*(sqrt(3)*r20*sin(th1) + r21*sin(th1) - 2*r22*cos(th1)) - 0.1615*f1*r20*(sqrt(3)*r10*sin(th1) + r11*sin(th1) - 2*r12*cos(th1)) + 0.08075*f2*(r10 - sqrt(3)*r11)*(sqrt(3)*r20*sin(th2) - r21*sin(th2) + 2*r22*cos(th2)) - 0.08075*f2*(r20 - sqrt(3)*r21)*(sqrt(3)*r10*sin(th2) - r11*sin(th2) + 2*r12*cos(th2)) + 0.1615*f3*(r10 + sqrt(3)*r11)*(r21*sin(th3) + r22*cos(th3)) - 0.1615*f3*(r20 + sqrt(3)*r21)*(r11*sin(th3) + r12*cos(th3))) + (0.082588877652288*r00*r10 + 0.0635392176642386*r01*r11 + 0.0367867310522326*r02*r12)*(0.1615*f1*r00*(sqrt(3)*r20*sin(th1) + r21*sin(th1) - 2*r22*cos(th1)) - 0.1615*f1*r20*(sqrt(3)*r00*sin(th1) + r01*sin(th1) - 2*r02*cos(th1)) + 0.08075*f2*(r00 - sqrt(3)*r01)*(sqrt(3)*r20*sin(th2) - r21*sin(th2) + 2*r22*cos(th2)) - 0.08075*f2*(r20 - sqrt(3)*r21)*(sqrt(3)*r00*sin(th2) - r01*sin(th2) + 2*r02*cos(th2)) + 0.1615*f3*(r00 + sqrt(3)*r01)*(r21*sin(th3) + r22*cos(th3

In [11]:
# Find angular position
w0x, w0y, w0z = sp.symbols('w0x w0y w0z')
theta0x, theta0y, theta0z = matrix2rpy(R)
w0 = sp.Matrix([w0x,w0y,w0z])
theta0 = sp.Matrix([theta0x,theta0y,theta0z])
w = w0 + sp.integrate(alpha,t)
w = sp.simplify(w)
print("Angular Velocity:")
display(w)
theta = theta0 + sp.integrate(w,t)
theta = sp.simplify(theta)
print("Angular Position:")
display(theta)

Angular Velocity:


Matrix([
[ -t*((0.082588877652288*r00**2 + 0.0635392176642386*r01**2 + 0.0367867310522326*r02**2)*(0.1615*f1*r10*(sqrt(3)*r20*sin(th1) + r21*sin(th1) - 2*r22*cos(th1)) - 0.1615*f1*r20*(sqrt(3)*r10*sin(th1) + r11*sin(th1) - 2*r12*cos(th1)) + 0.08075*f2*(r10 - sqrt(3)*r11)*(sqrt(3)*r20*sin(th2) - r21*sin(th2) + 2*r22*cos(th2)) - 0.08075*f2*(r20 - sqrt(3)*r21)*(sqrt(3)*r10*sin(th2) - r11*sin(th2) + 2*r12*cos(th2)) + 0.1615*f3*(r10 + sqrt(3)*r11)*(r21*sin(th3) + r22*cos(th3)) - 0.1615*f3*(r20 + sqrt(3)*r21)*(r11*sin(th3) + r12*cos(th3))) - (0.082588877652288*r00*r10 + 0.0635392176642386*r01*r11 + 0.0367867310522326*r02*r12)*(0.1615*f1*r00*(sqrt(3)*r20*sin(th1) + r21*sin(th1) - 2*r22*cos(th1)) - 0.1615*f1*r20*(sqrt(3)*r00*sin(th1) + r01*sin(th1) - 2*r02*cos(th1)) + 0.08075*f2*(r00 - sqrt(3)*r01)*(sqrt(3)*r20*sin(th2) - r21*sin(th2) + 2*r22*cos(th2)) - 0.08075*f2*(r20 - sqrt(3)*r21)*(sqrt(3)*r00*sin(th2) - r01*sin(th2) + 2*r02*cos(th2)) + 0.1615*f3*(r00 + sqrt(3)*r01)*(r21*sin(th3) + r22*cos

Angular Position:


Matrix([
[                   t**2*(-0.0102615836527745*f1*r00*r01*r11*r22*cos(th1) + 0.0102615836527745*f1*r00*r01*r12*r21*cos(th1) - 0.00297052853246779*f1*r00*r02*r11*r22*sin(th1) + 0.00297052853246779*f1*r00*r02*r12*r21*sin(th1) + 0.0102615836527745*f1*r01**2*r10*r22*cos(th1) - 0.0102615836527745*f1*r01**2*r12*r20*cos(th1) - 0.0102615836527745*f1*r01*r02*r10*r21*cos(th1) + 0.00297052853246779*f1*r01*r02*r10*r22*sin(th1) + 0.0102615836527745*f1*r01*r02*r11*r20*cos(th1) - 0.00297052853246779*f1*r01*r02*r12*r20*sin(th1) - 0.00297052853246779*f1*r02**2*r10*r21*sin(th1) + 0.00297052853246779*f1*r02**2*r11*r20*sin(th1) + 0.00666905187042226*sqrt(3)*f2*r00**2*r11*r22*cos(th2) - 0.00666905187042226*sqrt(3)*f2*r00**2*r12*r21*cos(th2) - 0.00666905187042226*sqrt(3)*f2*r00*r01*r10*r22*cos(th2) + 0.00513079182638727*f2*r00*r01*r11*r22*cos(th2) + 0.00666905187042226*sqrt(3)*f2*r00*r01*r12*r20*cos(th2) - 0.00513079182638727*f2*r00*r01*r12*r21*cos(th2) + 0.00666905187042226*sqrt(3)*f2*r00*r02*r10*r

# Function

In [13]:
thetaNew = theta.xreplace({t:t*0.5})
pNew = p.xreplace({rx:thetaNew[0], ry:thetaNew[1], rz:thetaNew[2]})
#pNew = sp.simplify(pNew)
display(pNew)

Matrix([
[p0x + t**2*(-0.521920668058455*f1*sin(th1)*sin(0.25*t**2*(-0.0102615836527745*f1*r00*r11**2*r22*cos(th1) + 0.0102615836527745*f1*r00*r11*r12*r21*cos(th1) - 0.00297052853246779*f1*r00*r11*r12*r22*sin(th1) + 0.00297052853246779*f1*r00*r12**2*r21*sin(th1) + 0.0102615836527745*f1*r01*r10*r11*r22*cos(th1) + 0.00297052853246779*f1*r01*r10*r12*r22*sin(th1) - 0.0102615836527745*f1*r01*r11*r12*r20*cos(th1) - 0.00297052853246779*f1*r01*r12**2*r20*sin(th1) - 0.0102615836527745*f1*r02*r10*r11*r21*cos(th1) - 0.00297052853246779*f1*r02*r10*r12*r21*sin(th1) + 0.0102615836527745*f1*r02*r11**2*r20*cos(th1) + 0.00297052853246779*f1*r02*r11*r12*r20*sin(th1) + 0.00666905187042226*sqrt(3)*f2*r00*r10*r11*r22*cos(th2) - 0.00666905187042226*sqrt(3)*f2*r00*r10*r12*r21*cos(th2) + 0.00513079182638727*f2*r00*r11**2*r22*cos(th2) - 0.00513079182638727*f2*r00*r11*r12*r21*cos(th2) - 0.00297052853246779*f2*r00*r11*r12*r22*sin(th2) + 0.00297052853246779*f2*r00*r12**2*r21*sin(th2) - 0.00666905187042226*sqrt(3)

# A, b for IPM

In [ ]:
f = sp.Function('f')
variables = 3
x = sp.Matrix([f1,f2,f3])
ptx, pty, ptz = sp.symbols('ptx pty ptz')
pt = sp.Matrix([ptx,pty,ptz])
f_vec = pNew-pt
f = f_vec.dot(f_vec)

print(f)

fd = sp.Matrix([f.diff(x[0]),f.diff(x[1]),f.diff(x[2])])
print(fd)
# Plot the function
#f_plot = f.xreplace({f3:1,ptx:1,pty:1,ptz:1,p0x:0,p0y:0,p0z:0,v0x:0,v0y:0,v0z:0,t:1,theta0x:0,theta0y:0,theta0z:0,w0x:0,w0y:0,w0z:0})
#sp.plotting.plot3d(f_plot, (f1, 0, 3), (f2, 0, 3))



1.08960473498634*(0.958*p0x - 0.958*ptx + t**2*(f1*sin(0.25*t**2*(-0.0102615836527745*f1*r00*r11**2*r22 + 0.0102615836527745*f1*r00*r11*r12*r21 + 0.0102615836527745*f1*r01*r10*r11*r22 - 0.0102615836527745*f1*r01*r11*r12*r20 - 0.0102615836527745*f1*r02*r10*r11*r21 + 0.0102615836527745*f1*r02*r11**2*r20 + 0.00666905187042226*sqrt(3)*f2*r00*r10*r11*r22 - 0.00666905187042226*sqrt(3)*f2*r00*r10*r12*r21 + 0.00513079182638727*f2*r00*r11**2*r22 - 0.00513079182638727*f2*r00*r11*r12*r21 - 0.00666905187042226*sqrt(3)*f2*r01*r10**2*r22 - 0.00513079182638727*f2*r01*r10*r11*r22 + 0.00666905187042226*sqrt(3)*f2*r01*r10*r12*r20 + 0.00513079182638727*f2*r01*r11*r12*r20 + 0.00666905187042226*sqrt(3)*f2*r02*r10**2*r21 - 0.00666905187042226*sqrt(3)*f2*r02*r10*r11*r20 + 0.00513079182638727*f2*r02*r10*r11*r21 - 0.00513079182638727*f2*r02*r11**2*r20 - 0.00666905187042226*sqrt(3)*f3*r00*r10*r11*r22 + 0.00666905187042226*sqrt(3)*f3*r00*r10*r12*r21 + 0.00513079182638727*f3*r00*r11**2*r22 - 0.00513079182638727*f

In [ ]:
fMax = sp.symbols('fMax')
fMin = sp.symbols('fMin')
maxConstraints = sp.Matrix([0]*variables)
minConstraints = sp.Matrix([0]*variables)
for i in range(1, variables+1):
    exec(f"maxConstraints[{i-1}] = fMax - f{i}")
    exec(f"minConstraints[{i-1}] = f{i} - fMin")
g = sp.Matrix([maxConstraints, minConstraints])
display(g)
print(g)

Matrix([
[-f1 + fMax],
[-f2 + fMax],
[-f3 + fMax],
[ f1 - fMin],
[ f2 - fMin],
[ f3 - fMin]])

Matrix([[-f1 + fMax], [-f2 + fMax], [-f3 + fMax], [f1 - fMin], [f2 - fMin], [f3 - fMin]])


In [ ]:
constraintsNum = len(g)
s = sp.Matrix([0]*constraintsNum)
l = sp.Matrix([0]*constraintsNum)
sln = sp.Matrix([0]*constraintsNum)
for i in range(constraintsNum):
    exec(f"s{i} = sp.symbols('s{i}')")
    s[i] = eval(f"s{i}")
    sln[i] = sp.ln(s[i])
    exec(f"l{i} = sp.symbols('l{i}')")
    l[i] = eval(f"l{i}")
mu = sp.symbols('mu')
# Unit vector
e = sp.Matrix([1]*constraintsNum)
lagrange = f - (mu * e.T * sln)[0] + ((g+s).T * l)[0]
print(lagrange)


l0*(-f1 + fMax + s0) + l1*(-f2 + fMax + s1) + l2*(-f3 + fMax + s2) + l3*(f1 - fMin + s3) + l4*(f2 - fMin + s4) + l5*(f3 - fMin + s5) - mu*log(s0) - mu*log(s1) - mu*log(s2) - mu*log(s3) - mu*log(s4) - mu*log(s5) + 1.08960473498634*(0.958*p0x - 0.958*ptx + t**2*(f1*sin(0.25*t**2*(-0.0102615836527745*f1*r00*r11**2*r22 + 0.0102615836527745*f1*r00*r11*r12*r21 + 0.0102615836527745*f1*r01*r10*r11*r22 - 0.0102615836527745*f1*r01*r11*r12*r20 - 0.0102615836527745*f1*r02*r10*r11*r21 + 0.0102615836527745*f1*r02*r11**2*r20 + 0.00666905187042226*sqrt(3)*f2*r00*r10*r11*r22 - 0.00666905187042226*sqrt(3)*f2*r00*r10*r12*r21 + 0.00513079182638727*f2*r00*r11**2*r22 - 0.00513079182638727*f2*r00*r11*r12*r21 - 0.00666905187042226*sqrt(3)*f2*r01*r10**2*r22 - 0.00513079182638727*f2*r01*r10*r11*r22 + 0.00666905187042226*sqrt(3)*f2*r01*r10*r12*r20 + 0.00513079182638727*f2*r01*r11*r12*r20 + 0.00666905187042226*sqrt(3)*f2*r02*r10**2*r21 - 0.00666905187042226*sqrt(3)*f2*r02*r10*r11*r20 + 0.00513079182638727*f2*r02*

In [ ]:
# Compute the gradient of the lagrange function
grad = sp.Matrix([lagrange.diff(i) for i in x])

In [ ]:
# Compute the hessian matrix of the lagrange function
hess = sp.Matrix([[grad[i].diff(j) for i in range(len(x))] for j in x])

In [ ]:
# Compute jacobian of g
jacob = sp.Matrix([[g[i].diff(x[j]) for j in range(variables)] for i in range(constraintsNum)])
display(jacob)

Matrix([
[-1,  0,  0],
[ 0, -1,  0],
[ 0,  0, -1],
[ 1,  0,  0],
[ 0,  1,  0],
[ 0,  0,  1]])

In [ ]:
# Diagonal matrix
S = sp.diag(*s)
Ep = sp.diag(*l)
I = sp.eye(constraintsNum)

In [ ]:
A = sp.zeros(variables+constraintsNum*2, variables+constraintsNum*2)
A[:variables, :variables] = hess
A[:variables, variables:variables+constraintsNum] = jacob.T
A[variables:variables+constraintsNum, :variables] = jacob
A[variables:variables+constraintsNum, variables+constraintsNum:variables+constraintsNum*2] = sp.eye(constraintsNum)
A[variables+constraintsNum:variables+constraintsNum*2, variables:variables+constraintsNum] = sp.eye(constraintsNum)
A[variables+constraintsNum:variables+constraintsNum*2, variables+constraintsNum:variables+constraintsNum*2] = S.inv()*Ep

#display(A)

In [ ]:
b = sp.zeros(variables+constraintsNum*2, 1)
b[:variables, 0] = grad
b[variables:variables+constraintsNum, 0] = g+s
b[variables+constraintsNum:variables+constraintsNum*2, 0] = l-mu*S.inv()*e
#display(b)

# To cpp

In [ ]:
def cseCpp(expr):
    sub_exprs, simplified_expr = sp.cse(expr)
    simplified_expr = simplified_expr[0]

    sub_exprs_matrix = sp.zeros(len(sub_exprs), 1)
    for i in range(len(sub_exprs)):
        sub_exprs_matrix[i] = sub_exprs[i][1]

    sub_expr_cpp = sp.ccode(sub_exprs_matrix, assign_to='x')

    # Iterate over lines in sub_expr_cpp, and add float type
    for i in range(len(sub_exprs)):
        sub_expr_cpp = sub_expr_cpp.replace(f'x[{i}]', f'float x{i}')

    simplified_expr_cpp = sp.ccode(simplified_expr, assign_to='result')

    return simplified_expr_cpp, sub_expr_cpp

In [ ]:
f_cpp, f_sub_cpp = cseCpp(f)
fd_cpp, fd_sub_cpp = cseCpp(fd)

In [ ]:
A_cpp, A_sub_cpp = cseCpp(A)

In [ ]:
b_cpp, b_sub_cpp = cseCpp(b)

In [ ]:
# Make C++ file
cpp_code = f"""
#include <cmath>
#include "Eigen/Dense"

using namespace Eigen;

// Define a helper function to assign variables
void assignVariables(const Vector3f& f, const VectorXf& s, const VectorXf& l, 
                     const Vector3f& pt, const Vector3f& p0, const Vector3f& v0, const Vector3f& w0, const Matrix3f& R,
                     float& f1, float& f2, float& f3,  
                     float& s0, float& s1, float& s2, float& s3, float& s4, float& s5,
                     float& l0, float& l1, float& l2, float& l3, float& l4, float& l5,
                     float& ptx, float& pty, float& ptz, float& p0x, float& p0y, float& p0z, 
                     float& v0x, float& v0y, float& v0z, float& w0x, float& w0y, float& w0z,
                     float& r00, float& r01, float& r02, float& r10, float& r11, float& r12,
                     float& r20, float& r21, float& r22) {{
    // Assigning variables
    f1 = f[0];
    f2 = f[1];
    f3 = f[2];
    s0 = s[0];
    s1 = s[1];
    s2 = s[2];
    s3 = s[3];
    s4 = s[4];
    s5 = s[5];
    l0 = l[0];
    l1 = l[1];
    l2 = l[2];
    l3 = l[3];
    l4 = l[4];
    l5 = l[5];
    ptx = pt[0];
    pty = pt[1];
    ptz = pt[2];
    p0x = p0[0];
    p0y = p0[1];
    p0z = p0[2];
    v0x = v0[0];
    v0y = v0[1];
    v0z = v0[2];
    w0x = w0[0];
    w0y = w0[1];
    w0z = w0[2];
    r00 = R(0,0);
    r01 = R(0,1);
    r02 = R(0,2);
    r10 = R(1,0);
    r11 = R(1,1);
    r12 = R(1,2);
    r20 = R(2,0);
    r21 = R(2,1);
    r22 = R(2,2);
}}

// Define the function to be optimized
float costFunction(Vector3f f, VectorXf s, VectorXf l,
                   Vector3f pt, Vector3f p0, Vector3f v0, Vector3f w0, Matrix3f R, 
                   float mu, float fMax, float fMin, float t){{
    // Declare variables
    float f1, f2, f3, s0, s1, s2, s3, s4, s5, l0, l1, l2, l3, l4, l5;
    float ptx, pty, ptz, p0x, p0y, p0z, v0x, v0y, v0z, w0x, w0y, w0z, r00, r01, r02, r10, r11, r12, r20, r21, r22;

    // Call helper function to assign variables
    assignVariables(f, s, l, pt, p0, v0, w0, R, 
                    f1, f2, f3, s0, s1, s2, s3, s4, s5, l0, l1, l2, l3, l4, l5,
                    ptx, pty, ptz, p0x, p0y, p0z, v0x, v0y, v0z, w0x, w0y, w0z, r00, r01, r02, r10, r11, r12, r20, r21, r22);

{f_sub_cpp}

    // Perform your calculations using the assigned variables
    float result;
{f_cpp}

    return result;
}}

// Define the derivative of the function to be optimized
Vector3f costDerivative(Vector3f f, VectorXf s, VectorXf l,
                   Vector3f pt, Vector3f p0, Vector3f v0, Vector3f w0, Matrix3f R, 
                   float mu, float fMax, float fMin, float t){{
    // Declare variables
    float f1, f2, f3, s0, s1, s2, s3, s4, s5, l0, l1, l2, l3, l4, l5; 
    float ptx, pty, ptz, p0x, p0y, p0z, v0x, v0y, v0z, w0x, w0y, w0z, r00, r01, r02, r10, r11, r12, r20, r21, r22;

    // Call helper function to assign variables
    assignVariables(f, s, l, pt, p0, v0, w0, R, 
                    f1, f2, f3, s0, s1, s2, s3, s4, s5, l0, l1, l2, l3, l4, l5,
                    ptx, pty, ptz, p0x, p0y, p0z, v0x, v0y, v0z, w0x, w0y, w0z, r00, r01, r02, r10, r11, r12, r20, r21, r22);

{fd_sub_cpp}

    // Perform your calculations using the assigned variables
    Vector3f result;
{fd_cpp}

    return result;
}}

// Define the b vector function for IPM
VectorXf bFunction(Vector3f f, VectorXf s, VectorXf l,
                   Vector3f pt, Vector3f p0, Vector3f v0, Vector3f w0, Matrix3f R, 
                   float mu, float fMax, float fMin, float t){{
    // Declare variables
    float f1, f2, f3, s0, s1, s2, s3, s4, s5, l0, l1, l2, l3, l4, l5; 
    float ptx, pty, ptz, p0x, p0y, p0z, v0x, v0y, v0z, w0x, w0y, w0z, r00, r01, r02, r10, r11, r12, r20, r21, r22;

    // Call helper function to assign variables
    assignVariables(f, s, l, pt, p0, v0, w0, R, 
                    f1, f2, f3, s0, s1, s2, s3, s4, s5, l0, l1, l2, l3, l4, l5,
                    ptx, pty, ptz, p0x, p0y, p0z, v0x, v0y, v0z, w0x, w0y, w0z, r00, r01, r02, r10, r11, r12, r20, r21, r22);

{b_sub_cpp}

    // Perform your calculations using the assigned variables
    VectorXf result(15);
{b_cpp}

    return result;
}}

// Define the A matrix function for IPM 
MatrixXf AFunction(Vector3f f, VectorXf s, VectorXf l,
                   Vector3f pt, Vector3f p0, Vector3f v0, Vector3f w0, Matrix3f R, 
                   float mu, float fMax, float fMin, float t){{
    // Declare variables
    float f1, f2, f3, s0, s1, s2, s3, s4, s5, l0, l1, l2, l3, l4, l5; 
    float ptx, pty, ptz, p0x, p0y, p0z, v0x, v0y, v0z, w0x, w0y, w0z, r00, r01, r02, r10, r11, r12, r20, r21, r22;

    // Call helper function to assign variables
    assignVariables(f, s, l, pt, p0, v0, w0, R, 
                    f1, f2, f3, s0, s1, s2, s3, s4, s5, l0, l1, l2, l3, l4, l5,
                    ptx, pty, ptz, p0x, p0y, p0z, v0x, v0y, v0z, w0x, w0y, w0z, r00, r01, r02, r10, r11, r12, r20, r21, r22);

{A_sub_cpp}

    // Perform your calculations using the assigned variables
    VectorXf result(225);
{A_cpp}

    MatrixXf resultMatrix = Map<MatrixXf>(result.data(), 15, 15);
    return resultMatrix;
}}
"""

hpp_code = f"""
#ifndef DRONE_MODEL_HPP
#define DRONE_MODEL_HPP

#include "Eigen/Dense"

using namespace Eigen;

// Define a helper function to assign variables
void assignVariables(const Vector3f& f, const VectorXf& s, const VectorXf& l, 
                     const Vector3f& pt, const Vector3f& p0, const Vector3f& v0, const Vector3f& w0, const Matrix3f& R,
                     float& f1, float& f2, float& f3,  
                     float& s0, float& s1, float& s2, float& s3, float& s4, float& s5,
                     float& l0, float& l1, float& l2, float& l3, float& l4, float& l5,
                     float& ptx, float& pty, float& ptz, float& p0x, float& p0y, float& p0z, 
                     float& v0x, float& v0y, float& v0z, float& w0x, float& w0y, float& w0z,
                     float& r00, float& r01, float& r02, float& r10, float& r11, float& r12,
                     float& r20, float& r21, float& r22);

// Define the function to be optimized
float costFunction(Vector3f f, VectorXf s, VectorXf l,
                   Vector3f pt, Vector3f p0, Vector3f v0, Vector3f w0, Matrix3f R, 
                   float mu, float fMax, float fMin, float t);

// Define the derivative of the function to be optimized
Vector3f costDerivative(Vector3f f, VectorXf s, VectorXf l,
                   Vector3f pt, Vector3f p0, Vector3f v0, Vector3f w0, Matrix3f R, 
                   float mu, float fMax, float fMin, float t);

// Define the b vector function for IPM
VectorXf bFunction(Vector3f f, VectorXf s, VectorXf l,
                   Vector3f pt, Vector3f p0, Vector3f v0, Vector3f w0, Matrix3f R, 
                   float mu, float fMax, float fMin, float t);

// Define the A matrix function for IPM 
MatrixXf AFunction(Vector3f f, VectorXf s, VectorXf l,
                   Vector3f pt, Vector3f p0, Vector3f v0, Vector3f w0, Matrix3f R, 
                   float mu, float fMax, float fMin, float t);

#endif
"""

with open("cpp/droneModel.cpp", "w") as file:
    file.write(cpp_code)

with open("cpp/droneModel.hpp", "w") as file:
    file.write(hpp_code)